# Generic Crew Scheduling Problem from Beasley and Cao (1996) and its extensions (deadheading and layovers) from Derigs and Schäfer (2014)


In [1]:
# export

import os
from IPython.display import display, Markdown
import pandas as pd

%run "./instance_solution_handling.py"

In [2]:
#export
import ipywidgets as widgets
from ipywidgets import VBox,HBox
from ipywidgets import Dropdown
from ipywidgets import interact, interactive, fixed, interact_manual

In [3]:
#export
def compute_variant_to_results(instance_folder, solution_folder):
       
    n_activities_to_range_n_crews = get_n_activities_to_range_n_crews()

    variants = (["base","dh", "dhl"])
     
    variant_to_results = {}
    variant_to_better_solutions = {}
    variant_to_wrongly_claimed_optimal = {}
    
    for variant in variants:
        variant_to_results[variant] = []
        variant_to_better_solutions [variant] = 0
        variant_to_wrongly_claimed_optimal [variant] = 0
       
        
        for n_activities, range_crews in n_activities_to_range_n_crews.items():
            for n_crews in range_crews:
                
                if variant == "base" and n_crews == 129:
                    continue
                
                feasible, obj = check_sol_from_file(instance_folder, solution_folder, variant, n_activities, n_crews)
                obj_ds = get_obj_ds(n_activities, n_crews, variant, instance_folder)
                claimed_opt_ds = claimed_optimal_ds(n_activities, n_crews, variant, instance_folder)
                variant_to_results[variant].append([n_activities, n_crews, obj, obj_ds, claimed_opt_ds, obj < obj_ds, (obj < obj_ds and claimed_opt_ds)])

                if obj < obj_ds:
                    variant_to_better_solutions [variant]  +=1
                    if claimed_opt_ds:
                        variant_to_wrongly_claimed_optimal [variant]+= 1


    return variant_to_results, variant_to_better_solutions, variant_to_wrongly_claimed_optimal


In [4]:
#export
def get_comparison_total(variant_to_better_solutions, variant_to_wrongly_claimed_optimal):
    number_better = 0
    for var, better_solutions in variant_to_better_solutions.items():
        number_better += better_solutions
    
    number_wrong_optima = 0
    for var, wrong_optima in variant_to_wrongly_claimed_optimal.items():
        number_wrong_optima += wrong_optima
    return number_better, number_wrong_optima

In [5]:
#export
def report_variant_solutions(instance_folder, solution_folder,  variant, selected_activities, out_widget = None):
    
    results = [] 
    if not out_widget:
        out_widget = widgets.Output()
        display(out_widget)
    else:
        out_widget.clear_output()
    
    for result_variant in variant_to_results[variant]:
        
        if result_variant[0] not in selected_activities:
            continue

        results.append(result_variant)
        
    with out_widget:
        
        display(Markdown( f"**Variant {dropdown_variant.value}** Number of solutions better than in DS: {variant_to_better_solutions[dropdown_variant.value]}, number of wrongly claimed optimal values in DS: {variant_to_wrongly_claimed_optimal[dropdown_variant.value]}"))

        display(Markdown(f'### Detailed Objective Function Values: ' )) 
        
        display(pd.DataFrame(results, columns = [ 'Activities', 'Crews', 'Objective', 'Objective_DS', 'ClaimedOptimal_DS', 'BetterThanDS', 'WronglyClaimedOptimalDS']).style.hide(axis='index'))



In [6]:
#export
def create_dropdown_variant():
    dropdown_variant = widgets.Dropdown(
                       description="Variant:",
                       options =["base","dh", "dhl"])

    def on_change_variant(sender):
        report_variant_solutions(instance_folder,solution_folder, dropdown_variant.value, select_activities.value, df_widget)
        report_single_solution(instance_folder, solution_folder, dropdown_variant.value, dropdown_activity.value, dropdown_crews.value, solution_print_widget)

    dropdown_variant.observe(on_change_variant, names=['value'])
    
    return dropdown_variant

In [7]:
#export
def create_select_activities():

    select_activities = widgets.SelectMultiple(
        options=list(range(50, 501,50)),
        value=[50],
        rows=10,
        description='# Activities',
        disabled=False
    )

    def on_change_select_activities(sender):
        report_variant_solutions(instance_folder,solution_folder, dropdown_variant.value, select_activities.value, df_widget)

    select_activities.observe(on_change_select_activities, names=['value'])

    return select_activities
#display(select_activities)

In [8]:
#export
def create_dropdown_activity():
    dropdown_activity = widgets.Dropdown(
                description="# Activities",
                options =[i for i in range(50, 501,50)])


    def on_change_act(sender):

            activity_value = dropdown_activity.value
            dropdown_crews.options=list(get_n_activities_to_range_n_crews()[activity_value])
            dropdown_crews.value =  dropdown_crews.options[0]
            report_single_solution(instance_folder, solution_folder, dropdown_variant.value, activity_value, dropdown_crews.options[0], solution_print_widget )


    dropdown_activity.observe(on_change_act, names=['value'])
    return dropdown_activity


In [9]:
#export
def create_dropdown_crews():
    dropdown_crews = widgets.Dropdown(
                description="# Crews",
                options = get_n_activities_to_range_n_crews()[dropdown_activity.value])

    def on_change_crews(sender):
            report_single_solution(instance_folder, solution_folder, dropdown_variant.value, dropdown_activity.value, dropdown_crews.value, solution_print_widget )

    dropdown_crews.observe(on_change_crews, names=['value'])
    
    return dropdown_crews


In [10]:
#report_variant_solutions(instance_folder,solution_folder, "base", select_activities.value)

In [11]:
#export
def report_single_solution(instance_folder, solution_folder, variant, n_activities, n_crews, out_widget = None):
    
    if not out_widget:
        out_widget = widgets.Output()
        display(out_widget)
    else:
        out_widget.clear_output()
    
        
    pairings, instance = get_pairings_from_file(instance_folder, solution_folder, variant, n_activities, n_crews)
    
    with out_widget:
    
        display(Markdown(f'## Solution for Instance: {instance.instance_name}, Variant {variant}, Number of Crews: {n_crews}'))

        is_feasible, objective = check_pairings(instance, pairings, n_crews)

        if is_feasible:

            display( Markdown(f"Solution is feasible, Objective function value: **{objective}**." ))
            display( Markdown(f"Objective function value in DS paper:  **{get_obj_ds(n_activities, n_crews, variant, instance_folder)}** (claimed to be optimal: {claimed_optimal_ds(n_activities, n_crews, variant, instance_folder)})" ))
        else: 
            display(Markdown( f"Solution is infeasible" ))

        display(Markdown(f'###  Detailed Solution:'))

        for pairing in pairings:
            display(Markdown(get_pairing_str(pairing, instance)))
            
        
    

In [12]:
instance_folder = "./instances"
solution_folder = "./optimal_solutions"

variant_to_results, variant_to_better_solutions, variant_to_wrongly_claimed_optimal = compute_variant_to_results(instance_folder, solution_folder)
number_better, number_wrong_optima = get_comparison_total(variant_to_better_solutions, variant_to_wrongly_claimed_optimal)


## Comparing the Optimal Solutions Found with our State-Expanded Network Model to those reported in (Derigs and Schäfer 2014) (DS)

In [14]:

display(Markdown( f"**Total:** Number of solutions better than in DS: {number_better}, number of wrongly claimed optimal values in DS: {number_wrong_optima}"))

display(Markdown(f'## Results per problem variant (base, dh, dhl)' ))


dropdown_variant = create_dropdown_variant()
display(dropdown_variant)




display (Markdown("## Select numbers of activities for objective function results"))

aggregated_report_widget = widgets.Output()
df_widget = widgets.Output()

select_activities = create_select_activities()
with aggregated_report_widget:

    display(select_activities)
    display(df_widget)

report_variant_solutions(instance_folder,solution_folder, dropdown_variant.value, select_activities.value, df_widget)


display(aggregated_report_widget)


single_solution_widget = widgets.Output()
solution_print_widget = widgets.Output()

dropdown_activity = create_dropdown_activity()
dropdown_crews = create_dropdown_crews()

with single_solution_widget:
    display (Markdown("## Select instance for which to display our full solution:"))

    display(dropdown_activity)
    

    display(dropdown_crews)
    
    display(solution_print_widget)

report_single_solution(instance_folder, solution_folder, dropdown_variant.value, dropdown_activity.value, dropdown_crews.value, solution_print_widget )


display(single_solution_widget)



**Total:** Number of solutions better than in DS: 92, number of wrongly claimed optimal values in DS: 27

## Results per problem variant (base, dh, dhl)

Dropdown(description='Variant:', options=('base', 'dh', 'dhl'), value='base')

## Select numbers of activities for objective function results

Output()

Output()